<a href="https://colab.research.google.com/github/Xixaus/Nastroje/blob/main/CD_extrakce%2Bvyhlazen%C3%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Pro spuštění skriptu je nutné se přihlásit do Google účtu.
2. Spusťte 1. buňku tlačítkem ▶ v levém horním rohu nebo kliknutím do buňky a stistnutím klávesové zkratky ctrl+enter aby se načetly knihovny a vytvořili složky.
3. Do složky Samples vložte soubory s vzorky a do složky blanks vložte soubory s blanky

    !!! Je nutné aby pH vzoku bylo v názvu v souboru v následujícím formátu: ... _ 7_05_pufr.txt, kdy tímto způsobem bude končit název souboru !!!

3. Spusťte 2. buňku pro vygenerování grafu a souboru
    
    !!! pokud !!!
4. Soubor data.txt vložte do Bestselu, pokud Vám bude hlásit chybu, zkopírujte všechny data ze souboru a vložte do okna v Bestselu

In [16]:
#@title Vstupní hodnoty { run: "auto", vertical-output: true }

file_version = "new" # @param ["old", "new"] {allow-input: true}


In [1]:
#@title Knihovny a vytvoření/smazání složek { run: "auto", vertical-output: true }

#načtení knihoven
import os
import shutil
import pandas as pd
from scipy.signal import savgol_filter
import plotly.graph_objects as go

#vytvoření potřebných složek pro data
if os.path.exists("/content/samples"):
    shutil.rmtree("/content/samples")
os.makedirs("/content/samples", exist_ok=True)

if os.path.exists("/content/blanks"):
    shutil.rmtree("/content/blanks")
os.makedirs("/content/blanks", exist_ok=True)

#smazání zbytečné složky
if os.path.exists("/content/sample_data"):
    shutil.rmtree("/content/sample_data")

In [46]:
#@title Výpočty { run: "auto", vertical-output: true }

#funkce pro extrakci dat ze souborů

def extract_data(folder_path, file_version = "old"):
    file_list = os.listdir(folder_path)
    df = pd.DataFrame() #vytvoření prázného dataframe

    for filename in file_list: #iterace každého souboru ve složce
        if filename.endswith(".txt"): #podmínka pro extrakci pouze z .txt souborů
            number = (filename.split("_")[-3].split(".")[0]) + "." + filename.split("_")[-2].split(".")[0] #pojmenování spektra, konkrétně pH, které je na konci ve formátu ..._7_05_pufr.txt


            if file_version == "old":
                axis = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[0], skiprows=19, header=None) #extrakce vlnových délek
                axis = axis.values.flatten()

                data = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[1], skiprows=19, header=None) # extrakce spektra
                data = data.rename(columns={1: float(number)}) #přejmenování spektra podle promněnné number
                data.index = axis

            else:
                axis = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[0], skiprows=20, header=None) #extrakce vlnových délek

                end = axis[axis == "##### Extended Information"].dropna(how='all').index.item()
                axis = axis.iloc[:end,].values.flatten()

                data = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[1], skiprows=20, header=None) # extrakce spektra
                data = data.iloc[:end,]
                data = data.rename(columns={1: float(number)}) #přejmenování spektra podle promněnné number
                data.index = axis

            df = pd.concat([df, data], axis=1) #spojení dataframe se spectrem

    df = df.sort_index(axis=1) #seřazení sepekter podle názvu (number)
    return(df)

samples = extract_data("/content/samples", file_version) #extrakce naměřených spekter
blanks = extract_data("/content/blanks") #extrakce blanků


spectra = samples - blanks
spectra_SG = samples - blanks #odečtení blanků od spekter

spectra = spectra.astype("float16")
spectra_SG = spectra_SG.astype("float16")

for column in spectra_SG.columns:
    spectra_SG[column] = savgol_filter(spectra_SG[column], window_length=7, polyorder=2, deriv=0) #vyhlazení pomocí savitzkého-golayových filtrů

fig1 = go.Figure() #vykreslení grafu

for column in spectra.columns:
    fig1.add_trace(go.Scatter(y=spectra[column], x=spectra.index, mode='lines', name=str(column)))

fig1.update_layout(title="Spektra")
fig1.show()

fig2 = go.Figure() #vykreslení grafu
display(spectra)


spectra.to_csv('spectra.txt', sep='\t') #uložení spekter do .txt souboru
spectra_SG.to_csv('spectra_vyhlazena.txt', sep='\t') #uložení spekter do .txt souboru

,6.05
"350,0000",NaN
"349,0000",NaN
"348,0000",NaN
"347,0000",NaN
"346,0000",NaN
...,...
"204,0000",NaN
"203,0000",NaN
"202,0000",NaN
"201,0000",NaN


In [ ]:
for column in spectra.columns:
    fig2.add_trace(go.Scatter(y=spectra_SG[column], x=spectra_SG.index, mode='lines', name=str(column)))

fig2.update_layout(title="Vyhlazená spektra")
fig2.show()

spectra_SG.astype("float16")

display(spectra_SG)

,2.06,3.00,4.20,5.03,6.02,7.10,8.07,9.02,10.10,10.98
300.0,0.094257,0.087976,0.042630,0.123430,-0.039238,-0.045712,0.043999,0.076971,0.096002,0.184079
299.0,0.093574,0.134913,0.077279,0.059003,0.034652,0.012588,0.070626,0.095313,0.086277,0.200995
298.0,0.090803,0.162859,0.096218,0.018914,0.082504,0.051505,0.088982,0.105511,0.077418,0.210752
297.0,0.085944,0.171815,0.099446,0.003165,0.104317,0.071041,0.099068,0.107562,0.069424,0.213351
296.0,0.086182,0.163518,0.085209,0.010330,0.084142,0.065077,0.103904,0.110754,0.047649,0.201832
...,...,...,...,...,...,...,...,...,...,...
204.0,-5.475987,-3.947086,-4.543787,-4.741712,-3.082955,-3.995065,-3.772528,-5.282390,-2.186848,-0.684863
203.0,-4.280850,-2.531680,-2.929027,-2.859178,-2.117199,-1.653826,-3.148654,-3.686034,0.009963,0.356305
202.0,-2.678555,-1.184554,-1.737974,-1.783816,-1.101021,-0.766142,-2.045787,-2.206218,0.462849,0.519920
201.0,-0.740569,0.248939,-0.725797,-0.892925,-0.206534,-0.671825,-0.639276,-0.508306,-0.000893,0.285260
